# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

In [13]:
# Import dependencies
from pymongo import MongoClient
from pymongo.server_api import ServerApi
from pprint import pprint
from atlascompass import mongopass
import pandas as pd

In [ ]:
# Create an instance of MongoClient
# mongo = MongoClient(port=27017)

## not using the local host due to issues with other services running on the same port.Instead, using the API to the free cloud service.

In [7]:
uri = "mongodb+srv://kuromasadev:{}@clusteraprda.x7kvjsy.mongodb.net/?retryWrites=true&w=majority".format(mongopass)
# Create a new client and connect to the server
mongo = MongoClient(uri, server_api=ServerApi('1'))
# Send a ping to confirm a successful connection
try:
    mongo.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

Pinged your deployment. You successfully connected to MongoDB!


In [10]:
# confirm that our new database was created

database_names = mongo.list_database_names()

if "uk_food" in database_names:
    print("The new database was created.")
else:
    print("The new database was not found.")

The new database was created.


In [8]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [11]:
# review the collections in our new database
collection_names = db.list_collection_names()

if collection_names:
    print("Collections in the new database:")
    for collection_name in collection_names:
        print(collection_name)
else:
    print("No collections found in the new database.")


Collections in the new database:
establishments


In [15]:
# review a document in the establishments collection
collection = db['establishments']

documents = collection.find()

document_preview = pd.DataFrame(documents)

document_preview.head(3)

,_id,FHRSID,ChangesByServerID,LocalAuthorityBusinessID,BusinessName,BusinessType,BusinessTypeID,AddressLine1,AddressLine2,AddressLine3,...,LocalAuthorityWebSite,LocalAuthorityEmailAddress,scores,SchemeType,geocode,RightToReply,Distance,NewRatingPending,meta,links
0,648000bb0191e0b16a204a17,254719,0,PI/000069980,Refreshment Kiosk,Restaurant/Cafe/Canteen,1,The Bay,St Margarets Bay,Kent,...,http://www.dover.gov.uk/,publicprotection@dover.gov.uk,"{'Hygiene': 0, 'Structural': 5, 'ConfidenceInM...",FHRS,"{'longitude': '1.387974', 'latitude': '51.1522...",,4587.347175,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'https://api.ratings...."
1,648000bb0191e0b16a204a18,1034540,0,PI/000078691,The Coastguard Inn,Pub/bar/nightclub,7843,The Bay,St Margarets Bay,Kent,...,http://www.dover.gov.uk/,publicprotection@dover.gov.uk,"{'Hygiene': 0, 'Structural': 5, 'ConfidenceInM...",FHRS,"{'longitude': '1.387974', 'latitude': '51.1522...",,4587.347175,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'https://api.ratings...."
2,648000bb0191e0b16a204a19,254250,0,PI/000066174,The Pines Calyx,Other catering premises,7841,The Pines Garden,Beach Road,St Margarets Bay,...,http://www.dover.gov.uk/,publicprotection@dover.gov.uk,"{'Hygiene': 0, 'Structural': 0, 'ConfidenceInM...",FHRS,"{'longitude': '1.383298', 'latitude': '51.1481...",,4587.362403,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'https://api.ratings...."


In [16]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [ ]:
# Create a dictionary for the new restaurant data


In [ ]:
# Insert the new restaurant into the collection


In [ ]:
# Check that the new restaurant was inserted


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [ ]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields


3. Update the new restaurant with the `BusinessTypeID` you found.

In [ ]:
# Update the new restaurant with the correct BusinessTypeID


In [ ]:
# Confirm that the new restaurant was updated


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [ ]:
# Find how many documents have LocalAuthorityName as "Dover"


In [ ]:
# Delete all documents where LocalAuthorityName is "Dover"


In [ ]:
# Check if any remaining documents include Dover


In [ ]:
# Check that other documents remain with 'find_one'


5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [ ]:
# Change the data type from String to Decimal for longitude and latitude


Use `update_many` to convert `RatingValue` to integer numbers.

In [ ]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

In [ ]:
# Change the data type from String to Integer for RatingValue


In [ ]:
# Check that the coordinates and rating value are now numbers


## Shutoff

In [17]:
mongo.close()